In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
from sagemaker import get_execution_role

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
%store -r bucket
%store -r training_job_name

### Load model from S3

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=f"s3://{bucket}/training_output/{training_job_name}/output/model.tar.gz",
    role=role,
    transformers_version="4.6",
    pytorch_version="1.7",
    py_version="py36",
)

### Deploy model

In [ ]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.p3.2xlarge"
)

### Load test dataset for inference requests

In [ ]:
import pandas as pd

In [ ]:
import botocore
from datasets.filesystems import S3FileSystem

s3_session = botocore.session.Session()
s3 = S3FileSystem(session=s3_session)
dataset_test = load_from_disk(f's3://{bucket}//processing_output/test_data/', fs=s3)
df_test = pd.DataFrame(dataset_test)

In [ ]:
df_test.head(10)

In [ ]:
ade_input= {"inputs": df_test.iloc[0]['text']}
predictor.predict(ade_input)

In [ ]:
ade_input= {"inputs": df_test.iloc[8]['text']}
predictor.predict(ade_input)

### Play with the model with made-up phrases

In [ ]:
ade_input= {"inputs": "I got a rash from taking aspirin"}
predictor.predict(ade_input)

In [ ]:
ade_input= {"inputs": "I watched football and got really excited"}
predictor.predict(ade_input)